In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName('recomm').getOrCreate()

In [3]:
data = spark.read.format('csv').load('/FileStore/tables/Machine_learning/movielens_ratings.csv', inferschema = True, header = True)

In [4]:
data.show(1)

In [5]:
data.describe().show()

In [6]:
train_data , test_data = data.randomSplit([0.8, 0.2])

In [7]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
model = als.fit(train_data)

In [9]:
predictions = model.transform(test_data)

In [10]:
predictions.show()

In [11]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [12]:
rmse = evaluator.evaluate(predictions)
rmse

In [13]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId', 'userId'])

In [14]:
single_user.show()

In [15]:
recommendations = model.transform(single_user)

In [16]:
recommendations.orderBy('prediction', ascending=False).show()